Generates and fills data model

In [134]:
import pandas as pd
import numpy as np
import iris
import time
import os
from sentence_transformers import SentenceTransformer

Get database cursor

In [135]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"{hostname}:{port}/{namespace}"
print(CONNECTION_STRING)
conn = iris.connect(CONNECTION_STRING, username, password)
cursor = conn.cursor()

localhost:1972/USER


In [136]:
vectorizer = SentenceTransformer('all-MiniLM-L6-v2')

searchPhraseHigh = "violent aggressor sexual assault rape knife crime stabbing slashing armed robbery mugging  weapons gun violence active shooter fatal assault"
searchPhraseMid = "pickpocket anti social behavior loitering public disorder mugging minor assault verbal harassment aggravated assault intimidation physical altercation"
searchPhraseLow = "house robbery shoplifting vandalism drug possession trespassing public nuisance petty theft loitering minor property crimes unauthorized entry"

tableName = "SafeRoute.CrimeDataSample"

# Define the SQL query with placeholders for the vector and limit
sql = f"""
    SELECT TOP ? ContextVector
    FROM {tableName}
    ORDER BY VECTOR_DOT_PRODUCT(ContextVector, TO_VECTOR(?)) DESC
    """

numberOfResults = 1000

searchVectorHigh = vectorizer.encode(searchPhraseHigh, normalize_embeddings=True).tolist()
cursor.execute(sql, [numberOfResults, str(searchVectorHigh)])
resultsHigh = cursor.fetchall()
print(resultsHigh[0])

searchVectorMid = vectorizer.encode(searchPhraseMid, normalize_embeddings=True).tolist()
cursor.execute(sql, [numberOfResults, str(searchVectorMid)])
resultsMid = cursor.fetchall()

searchVectorLow = vectorizer.encode(searchPhraseLow, normalize_embeddings=True).tolist()
cursor.execute(sql, [numberOfResults, str(searchVectorLow)])
resultsLow = cursor.fetchall()




('.031246185302734375,.072592616081237792968,.010110705159604549407,.048953980207443237304,.043148692697286605834,.054165691137313842773,-.073881894350051879882,-.036765091121196746826,.024719553068280220031,.035509612411260604858,.14885920286178588867,.031594730913639068603,-.055222362279891967773,.0035074085462838411331,.012729155831038951873,-.041387628763914108276,.053586483001708984375,-.047353100031614303588,-.057701397687196731567,.021937869489192962646,-.048309024423360824584,-.025210378691554069519,.092976197600364685058,-.016418244689702987671,-.096742212772369384766,.015070011839270591736,.033904366195201873779,.050540283322334289551,-.014054980129003524781,-.0055065308697521686553,.058694548904895782471,-.045476805418729782104,.048285871744155883789,.029118429869413375854,-.043028540909290313721,-.070719406008720397949,.078700229525566101074,.017791122198104858398,.074496507644653320312,-.024362862110137939453,.026343561708927154541,-.075216986238956451416,.0262953154742717

Train via keywords

In [137]:
# Convert each row's vector string into a list of floats, then to a NumPy array.
vector_list_high = [
    np.array([float(x) for x in row[0].split(',')], dtype=np.float32)
    for row in resultsHigh
]

vector_list_med = [
    np.array([float(x) for x in row[0].split(',')], dtype=np.float32)
    for row in resultsMid
]

vector_list_low = [
    np.array([float(x) for x in row[0].split(',')], dtype=np.float32)
    for row in resultsLow
]


# Stack the individual 1D arrays (vectors) into a 2D array.
transformedResultsH = np.vstack(vector_list_high)
transformedResultsM = np.vstack(vector_list_med)
transformedResultsL = np.vstack(vector_list_low)

transformedResults = np.vstack([transformedResultsH, transformedResultsM, transformedResultsL])

# transformedResults = np.array(results, dtype=np.float32)
print(transformedResults.shape)
print(transformedResults)


(3000, 384)
[[ 0.03124619  0.07259262  0.01011071 ...  0.00834085 -0.00866969
   0.10918556]
 [-0.00871359  0.10045722 -0.02093219 ... -0.02278677 -0.01483309
   0.0319922 ]
 [ 0.00700664  0.03747452 -0.03193188 ... -0.06719084  0.09332396
  -0.02541574]
 ...
 [ 0.06772064  0.01955954  0.06193124 ...  0.03419967 -0.06836893
   0.01571639]
 [-0.03351866  0.07536459 -0.04206793 ... -0.04804547 -0.03891229
   0.03464732]
 [-0.01685778  0.07231984  0.0577804  ...  0.01198181 -0.04248691
  -0.01522257]]


In [138]:
y_high = np.random.uniform(
  .6,
  1, 
  size=(transformedResultsH.shape[0])
  ).astype(np.float32)

y_mid = np.random.uniform(
  .4,
  .6, 
  size=(transformedResultsM.shape[0])
  ).astype(np.float32)

y_low = np.random.uniform(
  0,
  .4, 
  size=(transformedResultsL.shape[0])
  ).astype(np.float32)

y = np.concatenate([y_high, y_mid, y_low])

print(y)
print(y.shape)

[0.9518122  0.99835634 0.74591917 ... 0.28592157 0.37921515 0.28951836]
(3000,)


In [139]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [140]:
import xgboost

model = xgboost.XGBRegressor (
  objective = "reg:squarederror",
  n_estimators = 384,
  max_depth = 4,
  learning_rate = 0.1,
  random_state = 42
)

model.fit(transformedResults, y)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=384, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [143]:


# test = "sexual mugging with arson and blue jays"
# test = "walking steadily. Light. Boring."
test = "Man sexually assaults, stabs and kills woman outside nightclub".lower()

res = vectorizer.encode(test, normalize_embeddings=True)
print(res.shape)
res = np.array(res, dtype=np.float32).reshape(1, -1)
print(res.shape)

print(model.predict(res))

model.save_model("crime_score_regressor.JSON")




(384,)
(1, 384)
[0.9476356]
